In [1]:
import numpy as np
import pandas as pd
import os
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from nltk.corpus import stopwords
import keras.preprocessing.text as T
from keras.preprocessing.text import Tokenizer
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GanJinZERO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
print("Train shape : ",train.shape)
print("Test shape : ",test.shape)

Train shape :  (1306122, 3)
Test shape :  (56370, 2)


In [3]:
text_list_train = train["question_text"].values
text_list_test = test["question_text"].values
label_list_train = train["target"].values


train_len = 200000
# test_len = 1000
text_list_train = text_list_train[0:train_len]
# text_list_test = text_list_test[0:test_len]
label_list_train = label_list_train[0:train_len]


train_len = len(text_list_train)
test_len = len(text_list_test)
print(train_len)
print(test_len)

200000
56370


In [4]:
doc = np.concatenate((text_list_train, text_list_test))

In [5]:
vectorizer = TfidfVectorizer(stop_words='english', min_df=100, ngram_range=(1, 3))
# vectorizer = CountVectorizer(stop_words='english', min_df=100, ngram_range=(1, 3))
tfidf_model = vectorizer.fit(doc)
train_vector = tfidf_model.transform(text_list_train)
test_vector = tfidf_model.transform(text_list_test)

In [6]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(train_vector, label_list_train)
y_pred = mnb.predict(test_vector)

In [7]:
sum(y_pred)/len(y_pred)
out_df = pd.DataFrame({"qid":test["qid"].values})
out_df['prediction'] = y_pred
out_df.to_csv("submission.csv", index=False)